In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
from math import sqrt
import matplotlib
import matplotlib.pyplot as plt
import xlrd

In [2]:
routes = pd.read_excel("routes.xlsx")
constraints = pd.read_excel("data-demands-deliveyconstraint.xlsx")

routes.head()
routes = routes.rename(columns = {'Route#': 'Route'}, inplace = False)

constraints.head()
constraints = constraints.rename(columns = {'Stores': 'Store',
                                            'Demands (per week)': 'Demand', 
                                            'Delivery accepting constraints': 'Limit'}, inplace = False)

<ipython-input-2-28588434085d>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  routes = pd.read_excel("routes.xlsx")
<ipython-input-2-28588434085d>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  constraints = pd.read_excel("data-demands-deliveyconstraint.xlsx")


In [4]:
routes.stop1 = [x.strip("S") for x in routes.stop1]
routes.iloc[20:,2] = [x.strip("S") for x in routes.iloc[20:,2]]

routes

,Route,stop1,stop2
0,1,1,NaN
1,2,2,NaN
2,3,3,NaN
3,4,4,NaN
4,5,5,NaN
5,6,6,NaN
6,7,7,NaN
7,8,8,NaN
8,9,9,NaN
9,10,10,NaN


In [5]:
constraints

,Store,Demand,Limit
0,S1,194,64
1,S2,201,50
2,S3,169,28
3,S4,228,57
4,S5,237,47
5,S6,190,63
6,S7,166,26
7,S8,158,31
8,S9,209,69
9,S10,219,43


In [6]:
# Defining the model 
m = Model("Transportation")

Academic license - for non-commercial use only - expires 2021-03-22
Using license file /Users/jameschua/gurobi.lic


In [7]:
# Defining weekly demand 
d = constraints.iloc[:,1]

# Defining daily capacity limit
l = constraints.iloc[:,2]

In [19]:
# Declaring variables

# no. of routes (30), no. of stores (20), no. of days (7)
var = m.addVars(30, 20, 7)

In [20]:
# Setting the objective 
m.setObjective((100/50)*sum(var[i,j,k] for i in range(30) for j in range(20) for k in range(7)), GRB.MINIMIZE)

In [25]:
# Store weekly demand constraints 
for j in range(20):
    m.addConstr(sum(var[i,j,k] for i in range(30) for k in range(7)) == d[j])

# Truck daily capacity constraints 
for i in range(20): #for the first 20 routes... 
    for k in range(7):
        m.addConstrs(var[i,j,k] <= 50 for j in range(20))

for k in range(7): #for the next 10 routes, where there are >1 stores to deliver to
    for i in range(20,30):
        m.addConstr(sum(var[i,j,k] for j in [int(routes.stop1[i]),int(routes.stop2[i])]) <= 50)

# Store daily capacity constraints 
for k in range(7):
    for j in range(20):
        m.addConstr(sum(var[i,j,k] for i in range(30)) <= l[j])

# Route-Store pairings 
for i in range(20):  #first 20 routes 
    for k in range(7): # for all 7 days (because as long as the store is not in the route, doesn't matter which day)
        route_store = [int(routes.stop1[i])]
        for j in range(20):
            if j not in route_store: m.addConstr(var[i,j,k] == 0)
                
for i in range(20,30):
    for k in range(7): 
        route_stores = [int(routes.stop1[i]), int(routes.stop2[i])]
        for j in range(20):
            if j not in route_stores: m.addConstr(var[i,j,k] == 0)
    

# Quantity has to be non-negative 
m.addConstr(var[i,j,k] >= 0)
#
print(i)

19


In [22]:
int(routes.stop1[0])

1

In [23]:
# Solving the optimisation problem 

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 27832 rows, 8400 columns and 61072 nonzeros
Model fingerprint: 0xc43d94ca
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 26996 rows and 0 columns
Presolve time: 0.02s

Solved with dual simplex
Solved in 0 iterations and 0.02 seconds
Infeasible model


In [ ]:
# Printing the optimal solutions obtained 